In [74]:
@info("Loading packages")
using Pkg; Pkg.activate("../test")
using Plots, StatsPlots, Plots.Measures
using StatsBase
using Glob
using DataFrames, DataFramesMeta
using ProgressMeter
default(leg = false, lw = 1.5)

using ComponentArrays, StaticArrays
using Test
using Parameters
using Distributions
using OrdinaryDiffEq

# establishing type hierarchy
abstract type AbstractParams end
abstract type AbstractParamCollection end # an AbstractParamCollection contain a defined set of multiple AbstractParams instances
abstract type AbstractSpeciesParams <: AbstractParams end
abstract type AbstractGlobalODEParams <: AbstractParams end
abstract type AbstractABM end
abstract type AbstractAgent end

include("../src/DoseResponse.jl")
include("../src/Params.jl")
include("../src/Solvers.jl")
include("../src/StateVars.jl")
include("../src/IO.jl")
include("../src/ModelFunctions.jl")
include("../src/DEBODE.jl")
include("../src/ImpliedTraits.jl")
include("../src/Macros.jl")

┌ Info: Loading packages
└ @ Main c:\Users\simon\Documents\Julia\Hydra\test\abm.ipynb:1
  Activating project at `c:\Users\simon\Documents\Julia\Hydra\test`


@replicates

In [1]:
using Hydra

## ABM

In [78]:
using NamedTupleTools

In [3]:
struct Agent
    x::Float64
    y::Array{Float64}
end

In [14]:
using ComponentArrays

## Side quest: Are structs faster than mutable structs?

question relates to accessing deeple nested values inside the struct

In [15]:
struct NestedAgent
    u::ComponentVector
    du::ComponentVector
    p
end

In [23]:
n = NestedAgent(
    ComponentVector(x = 1),
    ComponentVector(x = 0),
    ODEParamCollection()
)

NestedAgent((x = 1), (x = 0), ODEParamCollection
  glb: GlobalODEParams
  spc: SpeciesParams
  agn: Nothing nothing
)

In [30]:
@benchmark glb = n.p.glb

BenchmarkTools.Trial: 10000 samples with 990 evaluations.
 Range (min … max):  44.444 ns … 104.545 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     45.051 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   45.527 ns ±   2.899 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

   ▃█                                                           
  ▃██▄█▅▆▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▂▂▂▁▁▁▂▂▁▂▂▂▁▁▂▁▂▂▁▂▁▂▂ ▂
  44.4 ns         Histogram: frequency by time         56.7 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [24]:
using BenchmarkTools
@benchmark n.p.glb.C_W

BenchmarkTools.Trial: 10000 samples with 982 evaluations.
 Range (min … max):  68.024 ns … 181.059 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     69.145 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   69.934 ns ±   3.924 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

    ▂█▁  ▃                                                      
  ▂▅███▄▃█▄▂▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▂▂▂▂▂▂▂▂▁▂▁▂▂▂▂▁▂▁▁▂▂▂▂▂▂▂▂▁▁▂▂▂▂ ▂
  68 ns           Histogram: frequency by time         86.5 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [34]:
p = n.p
@benchmark p.glb.C_W

BenchmarkTools.Trial: 10000 samples with 990 evaluations.
 Range (min … max):  49.293 ns … 247.677 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     50.202 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   51.182 ns ±   6.645 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▅█▆▅                     ▁                                   ▁
  ████▆▆██▇▆▆▆▆▅▅▅▅▆▄▅▅▅▅▅██▆▅▅▆▆▅▅▃▅▅▄▃▅▄▃▄▄▄▄▃▄▃▄▄▁▄▄▄▃▁▁▄▄▄ █
  49.3 ns       Histogram: log(frequency) by time        81 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [32]:
@benchmark glb.C_W

BenchmarkTools.Trial: 10000 samples with 996 evaluations.
 Range (min … max):  28.815 ns … 78.614 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     29.116 ns              ┊ GC (median):    0.00%
 Time  (mean ± σ):   29.272 ns ±  1.588 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

   ██                                                          
  ▃███▅▁▁▂▂▂▁▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▁▁▁▂▂▂▂▂▁▂▂▂▂▁▁▁▁▁▁▁▁▁▁▂ ▂
  28.8 ns         Histogram: frequency by time        35.1 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [5]:
a = Agent(1., [1., 2.])

Agent(1.0, [1.0, 2.0])

In [35]:
mutable struct MutableAgent
    u::ComponentVector
    du::ComponentVector
    p
end

In [37]:
m = MutableAgent(
    ComponentVector(x = 1),
    ComponentVector(x = 0),
    ODEParamCollection()
)

MutableAgent((x = 1), (x = 0), ODEParamCollection
  glb: GlobalODEParams
  spc: SpeciesParams
  agn: Nothing nothing
)

In [38]:
using BenchmarkTools
@benchmark m.p.glb.C_W

BenchmarkTools.Trial: 10000 samples with 975 evaluations.
 Range (min … max):  70.564 ns … 322.564 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     72.308 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   74.321 ns ±  11.116 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▅█▆▁▁                                                        ▁
  ██████▇▇▇▆▆▅▆▆▆▅▅▅▅▆▇▆▄▆▅▅▅▅▆▄▅▅▄▁▄▅▅▅▅▆▄▆▆▄▆▅▄▅▄▆▅▇▇▆▆▆▆▇▆▇ █
  70.6 ns       Histogram: log(frequency) by time       126 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [39]:
p = m.p
@benchmark p.glb.C_W

BenchmarkTools.Trial: 10000 samples with 989 evaluations.
 Range (min … max):  49.545 ns … 252.882 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     50.556 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   53.081 ns ±   9.048 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▆█▅▁                                                         ▁
  ████▇███▇▆▇▆▆▅▆▆▆▅▇▇▇▆▅▆▅▅▆▅▆▅▅▆▄▄▅▅▄▅▄▅▅▅▆▅▆▆▆▆▇▇▅▆▆▆▇▆▆▇█▇ █
  49.5 ns       Histogram: log(frequency) by time      90.7 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [42]:
using Parameters

In [45]:
@unpack u, du, p = m

MutableAgent((x = 1), (x = 0), ODEParamCollection
  glb: GlobalODEParams
  spc: SpeciesParams
  agn: Nothing nothing
)

In [2]:
abstract type AbstractABM end

"""
Currently assumes that all agents are represented by a single agent type `AgentType`.
"""
mutable struct ABM <: AbstractABM
    p::AbstractParamCollection # parameter collection
    t::Float64 # current simulation time
    dt::Float64 # timestep
    euler::Function # definition of the euler function for the given dt
    saveat::Float64
    
    u::ComponentVector # global state variables
    du::ComponentVector # global derivatives

    AgentID_count::Int64 # cumulative number of agents in the simulation
    agents::AbstractVector # population of agents
    aout::AbstractVector # agent output
    mout::AbstractVector # model output

    """
    Instantiate ABM from param collection `p`. 
    """
    function ABM(
        p::A; 
        dt = 1/24, 
        saveat = 1,
        execute = true
        ) where A <: AbstractParamCollection

        abm = new() # initialize ABM object

        abm.p = p # store the parameters
        abm.t = 0. # initialize simulation time
        abm.dt = dt # timestep is given as keyword argument
        abm.euler = defeuler(dt) # Euler function for the given dt
        abm.saveat = saveat
        
        abm.u = init_substates_global(p) # initialize the global substates
        abm.du = similar(abm.u) # initialize the global derivatives
        abm.du.X_p = 0.
        abm.du.C_W .= 0.
        
        abm.AgentID_count = 0 # set agent count
        initialize_agents!(abm) # initailize the population of agents
        abm.aout = [] # initialize agent output
        abm.mout = [] # initialize model output

        if execute
            run!(abm) # run the model

            return prepare_output(abm) # prepare output in dataframes and return
        else
            return abm
        end
    end
end

ABM

In [3]:
using Random

"""
step!(
    abm::ABM; 
    )

Execution of a generic ABM step, following the schedule: 

1. Shuffle agents
2. Calculate global derivatives
3. Calculate agent derivatives
4. Update agent states
5. Record agent states
6. Update global states
7. Record global states

It is important that the agent steps occur between calculation of the global derivatives and 
updating the global states, because global derivatives which may be influenced by agent derivatives 
are initialized during calculation of the global states and may be mutated by the agent step. <br>
Changing this order will lead to erroneous computation of the interaction between agents and the environment.
"""
function step!(abm::ABM)

    du, u, p = abm.du, abm.u, abm.p
    t = abm.t

    shuffle!(abm.agents)

    for func! in abm.p.glb.odefuncs # execute global ODE-based functions
        func!(du, u, p, t)
    end

    for func! in abm.p.glb.rulefuncs # execute global rule-based functions
        func!(abm)
    end
    
    for a in abm.agents # for every agent
        step!(a, abm) # execute agent steps
        record!(a, abm) # record agent data
    end

    map!(abm.euler, u, du, u) # apply the euler scheme to global states
    record!(abm) # record global states
    filter!(a -> a.u.agn.dead == false, abm.agents) # remove dead agents

    abm.t += abm.dt
end


step!

In [4]:
"""
    init_agents!(abm::AbstractABM)::nothing
Initialize the population of 
"""
function initialize_agents!(abm::AbstractABM)::Nothing

    abm.agents = [] # initialize a vector of agents with undefined values and defined length

    for i in 1:abm.p.glb.N0 # for the number of initial agents
        
        push!(abm.agents, abm.p.glb.AgentType(abm)) # initialize an agent and add it to the vector of agents
    end

    return nothing
end

initialize_agents!

## Agent

In [5]:
abstract type AbstractAgent end
import Base:copy


"""
The definition of a generic AbstractAgent.
"""
mutable struct DEBAgent <: AbstractAgent
    p::AbstractParamCollection
    u::ComponentVector
    du::ComponentVector
    AgentID::Int64

    """
        DEBAgent(abm::ABM)
    
    Initialization of a generic DEB agent within ABM structure.
    """
    function DEBAgent(abm::ABM)
        a = new()

        a.p = copy(abm.p) # parameters; TODO: #29 avoid copying everything
        a.p.agn = AgentParams(a.p.spc) # assign agent-level parameters (induces individual variability)
        initialize_statevars!(a, abm) # initialize agent-level state variables
        a.du = similar(a.u) # initialize agent-level derivatives
        a.du.glb = abm.du # reference to global derivatives
        a.du.agn = similar(a.u.agn) # derivatives of agent substates have same shape as states
        a.AgentID = abm.AgentID_count # assign AgentID
        abm.AgentID_count += 1 # increment AgentID counter

        aout_t = Vector{Any}(undef, length(abm.p.spc.adata)) #
        
        return a
    end
end

DEBAgent

In [23]:
abm = ABM(ABMParamCollection(); execute = false)
agent = DEBAgent(abm)

DEBAgent(ABMParamCollection
  glb: GlobalABMParams
  spc: SpeciesParams
  agn: AgentParams
, (agn = (X_emb = 19.42, S = 0.019420000000000003, S_0 = 0.019420000000000003, H = 0.0, H_b = 0.0, R = 0.0, f_X = 1.0, I_emb = 0.0, I_p = 0.0, I = 0.0, A = 0.0, M = 0.0, J = 0.0, Q = 0.0, D_G = [0.0], D_M = [0.0], D_A = [0.0], D_R = [0.0], D_h = [0.0], y_G = 1.0, y_M = 1.0, y_A = 1.0, y_R = 1.0, h_z = 0.0, h_S = 0.0, cum_repro = 1.0, age = 0.0, cohort = 0.0, dead = 0.0, causeofdeath = 0.0), glb = (X_p = 1200.0, C_W = [0.0], N_tot = 1.0)), (agn = (X_emb = 0.0, S = 0.0, S_0 = 0.0, H = 0.0, H_b = 0.0, R = 0.0, f_X = 0.0, I_emb = 0.0, I_p = 0.0, I = 0.0, A = 0.0, M = 0.0, J = 0.0, Q = 0.0, D_G = [0.0], D_M = [0.0], D_A = [0.0], D_R = [0.0], D_h = [0.0], y_G = 0.0, y_M = 0.0, y_A = 0.0, y_R = 0.0, h_z = 0.0, h_S = 0.0, cum_repro = 0.0, age = 0.0, cohort = 0.0, dead = 0.0, causeofdeath = 0.0), glb = (X_p = 0.0, C_W = [0.0], N_tot = 0.0)), 1)

In [6]:
"""
Update an agent's reference to global state variables.
"""
function update_glb!(agent::AbstractAgent, abm::AbstractABM)
    map!(x -> x, agent.u.glb, abm.u) 
end

"""
step!(agent::AbstractAgent, abm::AbstractABM; odefuncs::Vector{Function}, rulefuncs::Vector{Function})

Generic definition of agent step. \n
This definition is generic, so that the function body does not have to be modified 
in order to simulate different models.
"""
function step!(agent::AbstractAgent, abm::ABM)
    du, u, p = agent.du, agent.u, agent.p # unpack agent substates, derivatives and parameters
    t = abm.t

    update_glb!(agent, abm) # update references to global state variables
    
    for func! in agent.p.spc.rulefuncs # apply all rule-based functions
        func!(agent, abm)
    end

    for func! in agent.p.spc.odefuncs # apply all ODE-based functions
        func!(du, u, p, t) 
    end


    map!(abm.euler, u.agn, du.agn, u.agn) # apply the euler scheme to agent substates
end

"""
record!(agent::DEBAgent, abm::ABM)

Record agent output (only if `p.glb.recordagentvars == true`).
"""
function record!(agent::AbstractAgent, abm::ABM)
    if abm.p.glb.recordagentvars && isapprox(abm.t%abm.saveat, 0, atol = abm.dt)
        push!(
            abm.aout, 
            (
                t = abm.t, 
                AgentID = agent.AgentID, 
                u = map(x -> eval(:($agent.$(x))), agent.p.spc.adata)
                )
        )
    end
end

record!

In [7]:
"""
record!(abm::ABM)

Record model-level output.
"""
function record!(abm::ABM)
    if isapprox(abm.t%abm.saveat, 0, atol = abm.dt)
        push!(abm.mout, (t = abm.t, u = u = map(x -> eval(:(abm.$x)), abm.p.glb.adata)))
    end
end

"""
Run the ABM.
"""
function run!(abm::AbstractABM)
    while abm.t <= abm.p.glb.t_max
        step!(abm)
    end
end

run!

In [8]:
"""
    prepare_output(abm::AbstractABM)

Prepare ABM output, converting Vectors of ComponentArrays into DataFrames. 
Two separate DataFrames are returned containing global and agent-level state variables, respectively.
"""
function prepare_output(abm::AbstractABM)

    mout = DataFrame(hcat([[x.t] for x in abm.mout]...)', [:t]) |> 
    x-> hcat(x, DataFrame(hcat([m.u for m in abm.mout]...)', extract_colnames(abm.mout[1].u)))
    
    if abm.p.glb.recordagentvars
        aout = DataFrame(hcat([[x.t, x.AgentID] for x in abm.aout]...)', [:t, :AgentID]) |> 
        x-> hcat(x, DataFrame(hcat([a.u for a in abm.aout]...)', extract_colnames(abm.aout[1].u)))
    else
        aout = DataFrame()
    end

    return mout, aout
end


prepare_output

## Side quest: Are nested structs slower?

In [ ]:
using BenchmarkTools

mutable struct X1
    x::Float64
end

x = X1(1.)

@benchmark getproperty(x, :x)

BenchmarkTools.Trial: 10000 samples with 998 evaluations.
 Range (min … max):  21.343 ns …  5.375 μs  ┊ GC (min … max): 0.00% … 99.08%
 Time  (median):     22.745 ns              ┊ GC (median):    0.00%
 Time  (mean ± σ):   25.751 ns ± 72.503 ns  ┊ GC (mean ± σ):  4.20% ±  1.69%

  ▃▅██▆▆▅        ▁▂▄▄▅▅▄▄▃▂          ▁                        ▂
  ███████▅▇▅▅▆▆██████████████▇▇▇▇▇█▇▇██▆▇▇▅▆▄▄▁▃▃▅▃▄▆▅▃▅▅▆▆▅▅ █
  21.3 ns      Histogram: log(frequency) by time      42.5 ns <

 Memory estimate: 16 bytes, allocs estimate: 1.

In [ ]:
using BenchmarkTools

mutable struct X2
    x1::X1
end

x = X2(X1(1.))

@benchmark getproperty(x.x1, :x)

BenchmarkTools.Trial: 10000 samples with 993 evaluations.
 Range (min … max):  39.074 ns … 845.015 ns  ┊ GC (min … max): 0.00% … 92.13%
 Time  (median):     40.081 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   42.405 ns ±  15.522 ns  ┊ GC (mean ± σ):  0.54% ±  1.59%

  ▅█▇▄       ▁▂▁       ▁                                    ▁  ▁
  ████▇▇▇████████▆▆▆▆▆██▆▆▆▅▅▅▆▅▅▃▅▄▅▅▅▄▅▅▅▅▄▅▆▆▆▅▅▆▄▆▅▅▆▅▆▇██ █
  39.1 ns       Histogram: log(frequency) by time      69.7 ns <

 Memory estimate: 16 bytes, allocs estimate: 1.

Yes they are, about 1.5-times for a doubly nested struct.

## Side quest: Are nested component vectors slower?

In [ ]:
x = ComponentVector(x = 1.)

@benchmark getproperty(x, :x)

BenchmarkTools.Trial: 10000 samples with 951 evaluations.
 Range (min … max):   93.165 ns …  3.171 μs  ┊ GC (min … max): 0.00% … 94.61%
 Time  (median):      99.790 ns              ┊ GC (median):    0.00%
 Time  (mean ± σ):   106.061 ns ± 59.155 ns  ┊ GC (mean ± σ):  1.07% ±  1.90%

   ▇█▃         ▁                                                
  ▃███▇▄▃▃▂▂▃▅██▇▄▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  93.2 ns         Histogram: frequency by time          174 ns <

 Memory estimate: 32 bytes, allocs estimate: 2.

In [ ]:
x = ComponentVector(x = ComponentVector(x = 1.))

@benchmark x.x.x

BenchmarkTools.Trial: 10000 samples with 590 evaluations.
 Range (min … max):  200.339 ns …   6.719 μs  ┊ GC (min … max): 0.00% … 95.30%
 Time  (median):     222.373 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   239.811 ns ± 186.881 ns  ┊ GC (mean ± σ):  2.49% ±  3.14%

  ▁▄▃  ▆█▆▃▁         ▂▂▁                                        ▁
  ███████████▇█▇▇▇▇▆██████▇▆▆▇▇▆▆▆▆▆▅▆▆▅▆▄▅▅▆▇▆▆▆▆▅▆▅▆▆▆▆▇▆▇▇▇▇ █
  200 ns        Histogram: log(frequency) by time        404 ns <

 Memory estimate: 128 bytes, allocs estimate: 4.

In [ ]:
x = ComponentVector(x = ComponentVector(x = ComponentVector(x = 1.)))

@benchmark x.x.x.x

BenchmarkTools.Trial: 10000 samples with 271 evaluations.
 Range (min … max):  288.930 ns …  13.908 μs  ┊ GC (min … max): 0.00% … 96.21%
 Time  (median):     321.771 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   344.655 ns ± 362.404 ns  ┊ GC (mean ± σ):  2.91% ±  2.72%

  ▁▄▃   ▆██▅▂▁             ▂▃▃▂▁                                ▂
  ████▅████████▇▇▆▅▅▅▅▄▅▅▆███████▇▆▅▅▃▆▅▄▅▄▅▂▄▅▆▄▅▆▆▅▆▆▅▅▄▅▅▃▄▅ █
  289 ns        Histogram: log(frequency) by time        537 ns <

 Memory estimate: 224 bytes, allocs estimate: 6.

Yes they are, and the performance difference is even worse than for mutable structs...

In [ ]:
glb = GlobalODEParams()
spc = SpeciesParams()

odeparamcollection = (
    map(x -> eval(:($x = $glb.$x)), fieldnames(GlobalODEParams))...,
    map(x -> eval(:($x = $spc.$x)), fieldnames(SpeciesParams))...
)

(1, 21.0, 1200.0, 0.1, 0.05, [0.0], 1.0, Function[C_Wdot_const!, X_pdot_chemstat!], Dirac{Float64}(value=1.0), 1.0, 1.0, (X_emb_int = true, H_p = true, K_X = true), 19.42, 1.0, 22.9, 22.9, 0.539, 0.33, 0.8, 0.8, 0.95, 0.59, 0.504, 100.0, 0.5, 5.0, [0.0], [0.0], [0.0], [0.38], [0.0], Function[LL2], Function[LL2M], Function[LL2], Function[LL2], Function[LL2h], [1.0e10], [1.0e10], [1.0e10], [167.0], [1.0e10], [1.0e10], [1.0e10], [1.0e10], [0.93], [1.0e10], nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, Function[y_z!, h_S!, Idot!, Adot!, Mdot!, Jdot!, Sdot!, Hdot!, H_bdot!, Rdot!, Ddot!, age!], Function[reproduce_opportunistic!, die!], Expr[:(u.agn.S), :(u.agn.R), :(u.agn.H)])

In [ ]:
glbtuple = map(x -> eval(:($x = $glb.$x)), fieldnames(typeof(glb)))


(1, 21.0, 1200.0, 0.1, 0.05, [0.0], 1.0, Function[C_Wdot_const!, X_pdot_chemstat!])

In [ ]:
using BenchmarkTools

mutable struct X4
    x3::X3
end

x = X4(X3(X2(X1(1.))))

@benchmark getproperty(x.x3.x2.x1, :x)

BenchmarkTools.Trial: 10000 samples with 974 evaluations.
 Range (min … max):  76.283 ns … 915.708 ns  ┊ GC (min … max): 0.00% … 89.36%
 Time  (median):     78.439 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   82.042 ns ±  17.033 ns  ┊ GC (mean ± σ):  0.28% ±  1.53%

  ▅██▄▁▂▃▄▃▂▃▄▂        ▁▁                                      ▂
  ██████████████▇▇█▇▆▇███▆▆▅▅▇▅▆▆▆▆▆▆▆▅▅▅▅▅▅▅▅▅▅▄▄▄▄▄▄▄▄▄▅▅▅▄▅ █
  76.3 ns       Histogram: log(frequency) by time       134 ns <

 Memory estimate: 16 bytes, allocs estimate: 1.